In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import norm
import datetime as dt
timestamp = str(dt.datetime.now().strftime('%Y-%m-%d'))


In [ ]:
df_trem = pd.read_csv('/opt/rsam/my_rsam/MAVZ.asc', parse_dates=True, index_col=0, delim_whitespace=True, header=None, names=['tremor'])

In [ ]:
#drop Kaikoura EQ to remove large spike 2016_11_14

#2016-11-13T00:00:00.992598  +1.8256780097e+03
#2016-11-14T00:00:00.992769  +4.6793780794e+02
#2016-11-15T00:00:00.992940  +2.2306629279e+02
df_trem.drop(pd.to_datetime('2016-11-13 00:00:00.992598'), inplace=True)
df_trem.drop(pd.to_datetime('2016-11-14 00:00:00.992769'), inplace=True)
df_trem.drop(pd.to_datetime('2016-11-15 00:00:00.992940'), inplace=True)

df_trem.plot()
df_trem.describe()

In [ ]:
#read in datalogger temperature data
url = 'http://fits.geonet.org.nz/observation?siteID=RU001&typeID=t&days=6000&methodID=logic'
df = pd.read_csv(url, parse_dates=['date-time'], index_col=['date-time'], usecols = ['date-time', ' t (C)'])

#Read in the manual temperatures
#outlet, thermometer
url = 'http://fits.geonet.org.nz/observation?siteID=RU001&typeID=t&methodID=therm'
dfm = pd.read_csv(url, parse_dates=['date-time'], index_col=['date-time'], usecols = ['date-time', ' t (C)'])
df = df.append(dfm)

#outlet, thermocouple
url = 'http://fits.geonet.org.nz/observation?siteID=RU001&typeID=t&methodID=thermcoup'
dfm2 = pd.read_csv(url, parse_dates=['date-time'], index_col=['date-time'], usecols = ['date-time', ' t (C)'])
df = df.append(dfm2)

#centre, thermocouple
url = 'http://fits.geonet.org.nz/observation?siteID=RU003&typeID=t&methodID=thermcoup'
dfm3 = pd.read_csv(url, parse_dates=['date-time'], index_col=['date-time'], usecols = ['date-time', ' t (C)'])
df = df.append(dfm3)

#north, thermocouple
url = 'http://fits.geonet.org.nz/observation?siteID=RU004&typeID=t&methodID=thermcoup'
dfm4 = pd.read_csv(url, parse_dates=['date-time'], index_col=['date-time'], usecols = ['date-time', ' t (C)'])
df = df.append(dfm4)

#select dates of interest
df = df[df.index>='2013-01-01 00:00:00']

#resample all to daily value
df_temp_day = df.resample('D').mean()

#get rid of nans
df_temp_day.dropna(inplace=True)

#change annoying temp label and drop old column
df_temp_day['temp'] = df_temp_day[' t (C)']
df_temp_day.drop(' t (C)', axis=1, inplace=True)

In [ ]:
df_temp_day.plot()

In [ ]:
fig = plt.figure(figsize=(12,6))
ax = fig.add_subplot(111)
ax.plot(df_trem.index, df_trem.tremor, label='tremor')
ax.legend()
ax1 = ax.twinx()
ax.set_xlabel('year')
ax1.set_ylabel('temperature $^o$C', color='r')
ax.set_ylabel('tremor nm/s', color='b')
#ax1.plot(df_temp_day.index, np.gradient(df_temp_day.values.T[:][0]), color='red')
ax1.plot(df_temp_day.index, df_temp_day.values, color='red', label='temperature')
ax.text('2013-01-01', 950, timestamp)
ax1.legend(loc='upper left')

#ax1.set_xlim('2018','2019')
plt.title('MAVZ tremor and RCL temperature trends')
plt.savefig('/opt/html/Ruapehu_tremor_temp_overlay.png')


In [ ]:
df_trem_daily = df_trem.reindex(df_temp_day.index, method='nearest').interpolate()
df_trem_daily = df_trem_daily[df_trem_daily >= 0]

In [ ]:
from scipy.stats.stats import rankdata
from scipy.stats import spearmanr
plt.figure(figsize=(10,8))
r1 = rankdata(df_trem_daily.tremor)
r2 = rankdata(df_temp_day.values)
cbar = plt.hexbin(r1, r2, gridsize=25, cmap='plasma')
plt.scatter(r1, r2, s=2, c='w')
plt.colorbar(cbar, label='hexagon point density')
plt.xlabel('tremor rank')
plt.ylabel('temperature rank')
plt.title('ranked RCL temperature vs ranked tremor')
plt.text(0, 2000, timestamp, color='k')
plt.savefig('/opt/html/Ruapehu_tremor_temp.png')
spearmanr(df_trem_daily.tremor, df_temp_day.values)

This plot shows that tremor and temperature are weakly correlated.  Low temp lake tends to have low tremor and high temp lake tends to have high tremor

In [ ]:
#create probability histogram for all tremor values
dT = 25
Tmin = round(np.min(df_trem.values), -1)
Tmax = round(np.max(df_trem.values), 0)
bins = np.arange(Tmin, Tmax+dT, dT)
df_trem_hist = np.histogram(df_trem, bins = bins, density=True)

#plot histogram
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10,10), sharex=True)
ax1.hist(df_trem.tremor, bins = bins, density=True)
ax1.set_xlim(0,600)
ax1.grid()
ax1.set_ylabel('probability')
ax1.set_title('MAVZ tremor probability since 2013')
ax1.annotate('binwidth = 25 nm/s', xy=(200, 0.011))
ax1.text(500, 0.011, timestamp, color='k')

ax2.hist(df_trem.tremor, bins = bins, density=True, cumulative=True)
ax2.set_xlim(0,600)
ax2.grid()
ax2.set_yticks(np.arange(0, 1.1, 0.1))
ax2.set_xlabel('ground velocity nm/s')
ax2.set_ylabel('probability')


plt.tight_layout()
plt.savefig('/opt/html/Ruapehu_MAVZ_tremor_probability.png', dpi=300)

In [ ]:
RR_trem100 = len(df_trem[df_trem.tremor <=100])/df_trem.count()
RR_trem250 = len(df_trem[df_trem.tremor <=250])/df_trem.count()

print('tremor < 100 = %.4f' %RR_trem100.values)
print('tremor < 250 = %.4f' %RR_trem250.values)


